In [259]:
import sys
import netCDF4
import pickle
import numpy as np
import itertools
import pandas as pd

#######################

def extract_events1(p_data):
    def storm_def():
        def check_mits(mit):
            idx = (bin_events  <=-mit)*(bin_events<0)
            dry_periods = bin_events[idx]*-1
            return np.array([mit, np.std(dry_periods[1:]) / np.mean(dry_periods[1:]),
                                np.int32(np.mean(dry_periods[1:]))])

        v = (p_data==0)*1
        n = v==0
        a = ~n
        c = np.cumsum(a)
        d = np.diff(np.append([0.], c[n]))
        v[n] = -d
        dry_vec = np.cumsum(v)
        bin_events = np.append([1], np.diff(dry_vec))

        
        mit_dry = []
        for mit in mit_list:
            _mit = check_mits(mit)
            mit_dry.append(_mit)
            if ((_mit[1]<1.0) & (len(mit_dry)>1)):
                break    # condition satisfied
        mit_dry = np.array(mit_dry)
        min_mit = calc_min_mit(mit_dry)
        
        idx = (bin_events  <= -min_mit) * (bin_events < 0)
        dry_periods = bin_events[idx]*-1
        date_idx = np.where(idx[1:])[0]
        event_durations = date_idx[1:] -dry_periods[1:] - date_idx[0:-1]
        event_indices = date_idx[0:-1] + event_durations
        n_events = len(event_durations)
        # storm_def_new(data, mit)

        p_totals = [0]*n_events
        for i in range(n_events):
            p_totals[i] = np.sum(p_data[date_idx[i]:event_indices[i]+1])

        
        # summary = pd.DataFrame({'dt':dt_vec[date_idx[1:]], 't0':event_durations, 'tb':dry_periods[0:-1], 'intnesity':p_totals/event_durations})
        summary = [dt_vec[date_idx[1:]], event_durations, dry_periods[0:-1], p_totals/event_durations]
        return min_mit, summary
        # return np.array([min_mit, np.std(dry_periods[1:]) / np.mean(dry_periods[1:]), np.int32(np.mean(dry_periods[1:]))])

    #######################


    def calc_min_mit(p_events):
        idx, = np.where((np.diff(np.sign(p_events[:,1]-1)) != 0)*1==1)
        if len(idx)==0:
            a = np.abs(p_events[:,1]-1)
            idx, = np.where(a == a.min())
            return p_events[:,0][idx][0]
        return (p_events[:,0][idx][0]+p_events[:,0][idx+1][0])/2.0

    #######################

    result = np.empty((1,1), dtype=object)   
    n_data = len(p_data)
    # event_metrics = get_events(p_data, set_min_mit)
    # %timeit np.sum(p_data < 0)
    pos_idx = np.sum((p_data>=0) & (p_data<65535))

    # if float(np.sum(neg_idx)/n_data)<0.1:
    #     print('No data 1', float(np.sum(p_data < 0)/n_data))
    # p_data[p_data<0.5] = 0
        
    if (float(np.sum(pos_idx)/n_data)==0):
        result = np.array([np.nan,np.nan,np.nan])
        print('No data 2')
    else:
        # try:
        event_metrics = storm_def()
        result = event_metrics
        # except:
            # result = np.array([np.nan,np.nan,np.nan])
            # print('Try-except')

    # with open(fn_out_pickle, 'wb') as handle:
    #     pickle.dump(result, handle, protocol= pickle.HIGHEST_PROTOCOL)
    return result

######### INPUTS ##########

year = 2009

######### Global Variables #########

## Formatting variables
# fn_fmt = '/home/navid/Downloads/{year}_stage4_hourly.nc'
fn_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/precipitation/stage4/{year}_stage4_hourly.nc'
out_pickle_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/stage4_analysis/events/{year}.pickle'
# out_pickle_fmt = '/storage/home/hcoda1/6/njadidoleslam3/p-rbras6-0/projects/stochsm/stage4_analysis/events/{year}.pickle'


start_dt = '{year}-1-1'.format(year = year)
end_dt = '{year}-1-1'.format(year = year+1)
fn_nc_in = fn_fmt.format(year=year)
fn_out_pickle = out_pickle_fmt.format(year=year)
# technical variables
set_min_mit = 3
mit_list = [x for x in range(set_min_mit,12*24, 6)]


dt_vec = pd.date_range(start=start_dt, end=end_dt, freq='60min',closed='left')

dt_vec_month = dt_vec.month
# dt_vec = dt_vec

# read dataset row by row, i.e., 
f = netCDF4.Dataset(fn_nc_in)
# f.set_auto_maskandscale(False)
# f.set_auto_mask(True)

grid_y_list = list(range(881))
grid_x_list = list(range(1121))

# with open(fn_out_pickle, 'wb') as handle:
# grid_y, grid_x = (468, 439)   
# 104330475
# 104330471
# gridxy_list = [(471, 433)   ,(475, 433)]
gridxy_list = [(437, 79)]
# grid_y, grid_x = (437, 79)   
# for grid_y in grid_y_list:
data = dict()
for grid_xy in gridxy_list:
    grid_y, grid_x = grid_xy
    gid = int('1{gid_x}{gid_y}'.format(gid_x = str(grid_x).zfill(4), gid_y = str(grid_y).zfill(4)))
    data[gid] = np.array(f.variables['p01m'][:, grid_y , grid_x].data)


In [260]:
print(f.variables['lat'][grid_y , grid_x], f.variables['lon'][grid_y , grid_x])

38.44538882277874 -120.4139754786379


In [261]:
mit_list = [x for x in range(set_min_mit,10*24, 6)]

In [541]:
def storm_def(p_data):
    def calc_min_mit(p_events):
        idx, = np.where((np.diff(np.sign(p_events[:,1]-1)) != 0)*1==1)
        if len(idx)==0:
            a = np.abs(p_events[:,1]-1)
            idx, = np.where(a == a.min())
            return p_events[:,0][idx][0]
        return (p_events[:,0][idx][0]+p_events[:,0][idx+1][0])/2.0
    def check_mits(mit):
        idx = (bin_events  <=-mit)*(bin_events<0)
        dry_periods = bin_events[idx]*-1
        return np.array([mit, np.std(dry_periods[:]) / np.mean(dry_periods[:]),
                            np.int32(np.mean(dry_periods[:]))])

    v = (p_data==0)*1
    n = v==0
    a = ~n
    c = np.cumsum(a)
    d = np.diff(np.append([0.], c[n]))
    v[n] = -d
    dry_vec = np.cumsum(v)
    bin_events = np.append([1], np.diff(dry_vec))

    
    mit_dry = []
    for mit in mit_list:
        _mit = check_mits(mit)
        mit_dry.append(_mit)
        if ((_mit[1]<1.0) & (len(mit_dry)>1)):
        # if ( (len(mit_dry)>1)):
            break    # condition satisfied
    mit_dry = np.array(mit_dry)
    min_mit = calc_min_mit(mit_dry)
    print(min_mit)
    
    idx = (bin_events  <= -min_mit) * (bin_events < 0)
    dry_periods = bin_events[idx]*-1
    date_idx = np.where(idx[1:])[0]
    event_durations = date_idx[1:] -dry_periods[1:] - date_idx[0:-1]
    event_indices = date_idx[0:-1] + event_durations
    n_events = len(event_durations)
    # storm_def_new(data, mit)

    p_totals = [0]*n_events
    for i in range(n_events):
        p_totals[i] = np.sum(p_data[date_idx[i]:event_indices[i]+1])

    
    summary = pd.DataFrame({'dt':dt_vec[date_idx[0:-1]], 't0':event_durations, 'tb':dry_periods[0:-1], 'intensity':p_totals/event_durations})
    # return [dt_vec[date_idx[1:]], event_durations, dry_periods[0:-1], p_totals/event_durations]
    return summary, mit_dry
summary = dict()
mit_min = dict()
i=3
dt_vec = pd.date_range(start=start_dt, end=end_dt, freq='60min',closed='left')
for gid in list(data.keys()):
    # data[gid][data[gid]<1] = 0
    summary[gid], mit_min[gid] = storm_def(data[gid])

120.0


In [542]:
def calc_min_mit(p_events):
    idx, = np.where((np.diff(np.sign(p_events[:,1]-1)) != 0)*1==1)
    if len(idx)==0:
        a = np.abs(p_events[:,1]-1)
        idx, = np.where(a == a.min())
        return p_events[:,0][idx][0]
    return (p_events[:,0][idx][0]+p_events[:,0][idx+1][0])/2.0
def check_mits(mit):
    idx = (bin_events  <=-mit)*(bin_events<0)
    dry_periods = bin_events[idx]*-1
    return np.array([mit, np.std(dry_periods[:]) / np.mean(dry_periods[:]),
                        np.int32(np.mean(dry_periods[:]))])

p_data = data[gid]
v = (p_data==0)*1
n = v==0
a = ~n
c = np.cumsum(a)
d = np.diff(np.append([0.], c[n]))
v[n] = -d
dry_vec = np.cumsum(v)
bin_events = np.append([1], np.diff(dry_vec))


# mit_dry = []
# for mit in mit_list:
#     _mit = check_mits(mit)
#     mit_dry.append(_mit)
#     if ((_mit[1]<1.0) & (len(mit_dry)>1)):
#     # if ( (len(mit_dry)>1)):
#         break    # condition satisfied
# mit_dry = np.array(mit_dry)
# min_mit = calc_min_mit(mit_dry)
# print(min_mit)

# idx = (bin_events  <= -min_mit) * (bin_events < 0)
# dry_periods = bin_events[idx]*-1
# date_idx = np.where(idx[1:])[0]
# event_durations = date_idx[1:] -dry_periods[1:] - date_idx[0:-1]
# event_indices = date_idx[0:-1] + event_durations
# n_events = len(event_durations)
# storm_def_new(data, mit)

# p_totals = [0]*n_events
# for i in range(n_events):
#     p_totals[i] = np.sum(p_data[date_idx[i]:event_indices[i]+1])

In [832]:
def extract_events1(p_data):
    # %%timeit
    def storm_def():
        def calc_min_mit_new(p_events):
            idx, = np.where((np.diff(np.sign(p_events[:, 1]-1)) != 0)*1 == 1)
            if len(idx) == 0:
                a = np.abs(p_events[:, 1]-1)
                idx, = np.where(a == a.min())
                return p_events[:, 0][idx][0]
            return (p_events[:, 0][idx][0]+p_events[:, 0][idx+1][0])/2.0


        def check_mits_new(mit):
            idx = (bin_events_month <= -mit)*(bin_events_month < 0)
            dry_periods = bin_events_month[idx]*-1
            return np.array([mit, np.std(dry_periods[:]) / np.mean(dry_periods[:]),
                            np.int32(np.mean(dry_periods[:]))])


        
        p_data[-1] = 1
        v = (p_data == 0)*1
        n = v == 0
        a = ~n
        c = np.cumsum(a)
        d = np.diff(np.append([0.], c[n]))
        v[n] = -d
        dry_vec = np.cumsum(v)
        bin_events = np.append([1], np.diff(dry_vec))

        event_idx = bin_events < 0

        result = []
        for m in range(1,13):
            mnth_idx = dt_vec.month == m
            bin_events_month = bin_events[event_idx & mnth_idx]
            # CONDITION 1
            if (len(bin_events_month) == 0):
                # result.append([min_mit, np.nan, np.nan, np.nan])
                continue

            # CONDITION 1
            if m < 12:
                idx_next_events = np.where((dt_vec.month > m) & (event_idx))

                bin_events_next_events = bin_events[idx_next_events]
                bin_events_month = np.append(
                    bin_events_month, bin_events[idx_next_events[0][0]])

                mit_dry = []
                for mit in mit_list:
                    _mit = check_mits_new(mit)
                    mit_dry.append(_mit)
                    if ((_mit[1] < 1.0) & (len(mit_dry) > 1)):
                        # if ( (len(mit_dry)>1)):
                        break    # condition satisfied

                mit_dry = np.array(mit_dry)
                min_mit = calc_min_mit_new(mit_dry)

                idx = (bin_events <= -min_mit) * (bin_events < 0) * (dt_vec.month == m)
                date_idx = np.where(idx)[0]

                date_idx_month = np.append(date_idx, idx_next_events[0][0])
                dry_periods = bin_events[date_idx_month]*-1

                event_durations = date_idx_month[1:] - \
                    dry_periods[1:] - date_idx_month[0:-1]
                event_indices = date_idx_month[0:-1] + event_durations
                n_events = len(event_durations)
                # storm_def_new(data, mit)

                p_totals = [0]*n_events
                for i in range(n_events):
                    p_totals[i] = np.sum(p_data[date_idx[i]:event_indices[i]+1])
                result.append([[m]*n_events, event_durations,
                            dry_periods[0:-1], p_totals/event_durations])
            if m == 12:
                if (len(bin_events_month) == 1):
                    continue
                mit_dry = []
                for mit in mit_list:
                    _mit = check_mits_new(mit)
                    mit_dry.append(_mit)
                    if ((_mit[1] < 1.0) & (len(mit_dry) > 1)):
                        break    # condition satisfied
                mit_dry = np.array(mit_dry)
                min_mit = calc_min_mit_new(mit_dry)

                idx = (bin_events <= -min_mit) * (bin_events < 0) * (dt_vec.month == m)
                date_idx_month = np.where(idx)[0]
                dry_periods = bin_events[date_idx_month]*-1

                date_idx = np.where(idx)[0]
                event_durations = date_idx[1:] - dry_periods[1:] - date_idx[0:-1]
                event_indices = date_idx[0:-1] + event_durations
                n_events = len(event_durations)
                # storm_def_new(data, mit)

                p_totals = [0]*n_events
                for i in range(n_events):
                    p_totals[i] = np.sum(p_data[date_idx[i]:event_indices[i]+1])

                result.append([[m]*n_events, event_durations,
                            dry_periods[0:-1], p_totals/event_durations])

        return result
    #######################

    # result = np.empty((1,1), dtype=object)   
    n_data = len(p_data)
    pos_idx = np.sum((p_data>=0) & (p_data<65535))
        
    if (float(np.sum(pos_idx)/n_data)==0):
        result = np.array([np.nan,np.nan,np.nan])
        print('No data 2')
    else:
        # try:  
        result = storm_def()
            # result = event_metrics
        # except:
            # None

    # with open(fn_out_pickle, 'wb') as handle:
    #     pickle.dump(result, handle, protocol= pickle.HIGHEST_PROTOCOL)
    return result

In [860]:
import sys
import netCDF4
import pickle
import numpy as np
import pandas as pd

def extract_events1(p_data):
    # %%timeit
    def storm_def():
        def calc_min_mit_new(p_events):
            idx, = np.where((np.diff(np.sign(p_events[:, 1]-1)) != 0)*1 == 1)
            if len(idx) == 0:
                a = np.abs(p_events[:, 1]-1)
                idx, = np.where(a == a.min())
                return p_events[:, 0][idx][0]
            return (p_events[:, 0][idx][0]+p_events[:, 0][idx+1][0])/2.0


        def check_mits_new(mit):
            idx = (bin_events_month <= -mit)*(bin_events_month < 0)
            dry_periods = bin_events_month[idx]*-1
            return np.array([mit, np.std(dry_periods[:]) / np.mean(dry_periods[:]),
                            np.int32(np.mean(dry_periods[:]))])


        # One-time operations for dry-wet periods        
        p_data[-1] = 1
        v = (p_data == 0)*1
        n = v == 0
        a = ~n
        c = np.cumsum(a)
        d = np.diff(np.append([0.], c[n]))
        v[n] = -d
        dry_vec = np.cumsum(v)
        bin_events = np.append([1], np.diff(dry_vec))

        event_idx = bin_events < 0

        # Initialize the result array
        result = []

        # Loop over each month
        for m in range(1,13):
            mnth_idx = dt_vec == m
            bin_events_month = bin_events[event_idx & mnth_idx]
            # CONDITION 1: If no events, skip the month
            if (len(bin_events_month) == 0):
                # DO NOTHING
                continue

            # CONDITION 2: For Jan - November
            if m < 12:
                idx_next_events = np.where((dt_vec > m) & (event_idx))

                bin_events_next_events = bin_events[idx_next_events]
                bin_events_month = np.append(
                    bin_events_month, bin_events[idx_next_events[0][0]])

                mit_dry = []
                for mit in mit_list:
                    _mit = check_mits_new(mit)
                    mit_dry.append(_mit)
                    if ((_mit[1] < 1.0) & (len(mit_dry) > 1)):
                        break    # condition satisfied

                mit_dry = np.array(mit_dry)
                min_mit = calc_min_mit_new(mit_dry)

                idx = (bin_events <= -min_mit) * (bin_events < 0) * (dt_vec == m)
                date_idx = np.where(idx)[0]

                date_idx_month = np.append(date_idx, idx_next_events[0][0])
                dry_periods = bin_events[date_idx_month]*-1

                event_durations = date_idx_month[1:] - \
                    dry_periods[1:] - date_idx_month[0:-1]
                event_indices = date_idx_month[0:-1] + event_durations
                n_events = len(event_durations)

                p_totals = [0]*n_events
                for i in range(n_events):
                    p_totals[i] = np.sum(p_data[date_idx[i]:event_indices[i]+1])
                result.append([[m]*n_events, event_durations,
                            dry_periods[0:-1], p_totals/event_durations])
            
            # CONDITION 3: For December
            if m == 12:
                if (len(bin_events_month) == 1):
                    continue
                mit_dry = []
                for mit in mit_list:
                    _mit = check_mits_new(mit)
                    mit_dry.append(_mit)
                    if ((_mit[1] < 1.0) & (len(mit_dry) > 1)):
                        break    # condition satisfied
                mit_dry = np.array(mit_dry)
                min_mit = calc_min_mit_new(mit_dry)

                idx = (bin_events <= -min_mit) * (bin_events < 0) * (dt_vec == m)
                date_idx_month = np.where(idx)[0]
                dry_periods = bin_events[date_idx_month]*-1

                date_idx = np.where(idx)[0]
                event_durations = date_idx[1:] - dry_periods[1:] - date_idx[0:-1]
                event_indices = date_idx[0:-1] + event_durations
                n_events = len(event_durations)


                p_totals = [0]*n_events
                for i in range(n_events):
                    p_totals[i] = np.sum(p_data[date_idx[i]:event_indices[i]+1])

                result.append([[m]*n_events, event_durations,
                            dry_periods[0:-1], p_totals/event_durations])

        return result
    #######################

    n_data = len(p_data)
    pos_idx = np.sum((p_data>=0) & (p_data<65535))
        
    if (float(np.sum(pos_idx)/n_data)==0):
        events = np.array([np.nan,np.nan,np.nan])
        print('No data 2')
    else:
        try:  
            events = storm_def()
        except:
            None
    return events

######### INPUTS ##########
# year = int(sys.argv[1])
year = 2001
######### INPUTS ##########

######### Global Variables #########
## Formatting variables
fn_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/precipitation/stage4/{year}_stage4_hourly.nc'
out_pickle_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/stage4_analysis/events_mbased/{year}_new.pickle'

year1= year+1
start_dt = '{year}-1-1'.format(year = year)
end_dt = '{year}-1-1'.format(year = year + 1)
fn_nc_in = fn_fmt.format(year=year)
fn_out_pickle = out_pickle_fmt.format(year=year)

#### variables for defining the events
set_min_mit = 3
mit_list = [x for x in range(set_min_mit,12*24, 6)]
####

### Date vector converted to month index array
dt_vec = pd.date_range(start=start_dt, end=end_dt, freq='60min',closed='left')
dt_vec = dt_vec.month

    # read dataset row by row, i.e., 
f = netCDF4.Dataset(fn_nc_in)
    # f.set_auto_maskandscale(False)
    # f.set_auto_mask(True)

grid_y_list = list(range(881))[300:301]
grid_x_list = list(range(1121))[300:301]
######### Global Variables #########

with open(fn_out_pickle, 'wb') as handle:
    
    for grid_y in grid_y_list:
        data = np.array(f.variables['p01m'][:, grid_y , :].data)
        lol = []
        for grid_x in grid_x_list:
            gid = int('1{gid_x}{gid_y}'.format(gid_x = str(grid_x).zfill(4), gid_y = str(grid_y).zfill(4)))
            events = extract_events1(data[:,grid_x])
            lol.append([gid,year,] + events)
        pickle.dump(lol, handle, protocol= pickle.HIGHEST_PROTOCOL)


In [861]:
lol1 = []
lol1.append(lol)

In [862]:
lol1

[[[103000300,
   2001,
   [[1, 1, 1, 1, 1, 1, 1],
    array([17, 34, 34, 24, 24, 55, 48]),
    array([203,  33,  62,  14,  96,  48,  17]),
    array([0.26352941, 0.11823529, 0.10117647, 0.0575    , 0.19      ,
           0.09454545, 0.05      ])],
   [[2, 2, 2, 2, 2, 2, 2],
    array([ 1,  8,  1,  5, 13, 27,  8]),
    array([202,  14, 121,  13,   6, 261,  21]),
    array([3.88      , 0.56      , 3.68      , 0.07      , 0.07      ,
           0.12222222, 0.49625   ])],
   [[3, 3, 3],
    array([ 3, 72, 17]),
    array([180,  28, 102]),
    array([0.81      , 0.04361111, 0.18882353])],
   [[4, 4, 4, 4, 4, 4, 4],
    array([42, 24,  1,  1, 15,  1, 15]),
    array([462,  73, 194,  19,  31, 103,  26]),
    array([0.32095238, 0.11      , 0.23      , 0.23      , 0.18266667,
           3.91      , 0.262     ])],
   [[5, 5, 5],
    array([64, 58, 18]),
    array([101, 179, 104]),
    array([0.02546875, 0.14706897, 0.38      ])],
   [[6, 6], array([24, 23]), array([798,  59]), array([0.01, 0.08]

In [845]:
_events_all = []
# for i_y in range(len(lol)): # loop over grid_y
#     for i_x in range(len(lol[i_y])): # loop over grid_x
grid_xy = int(lol[0][0])
idx = lol[0][3][0] == 1
n_events = sum(idx)
if n_events >0:
    event_dur = lol[0][3][1][idx]
    dry_vec = lol[0][3][2][idx]
    intensity = lol[0][3][3][idx]
    for i in range(n_events):
        _events_all.append([grid_xy, event_dur[i],intensity[i],dry_vec[i]])

TypeError: 'bool' object is not iterable

In [854]:
np.array(lol[0][2][0])

array([1, 1, 1, 1, 1, 1, 1])

In [830]:
p_data1 = p_data.copy()
p_data1[-745:] = 0
result1= extract_events1(data[gid])

In [831]:
result1

[[[1, 1, 1],
  array([ 48,  24, 144]),
  array([ 13,  48, 360]),
  array([0.38      , 0.09      , 0.68666667])],
 [[2, 2, 2],
  array([ 48, 240, 144]),
  array([216,  24,  72]),
  array([0.205, 0.56 , 0.465])],
 [[3, 3, 3, 3],
  array([46, 48, 24, 48]),
  array([ 56,  18, 216, 120]),
  array([2.87173913, 0.405     , 0.03      , 0.7       ])],
 [[4, 4, 4, 4],
  array([ 96,  24,  48, 144]),
  array([360,  72, 192, 120]),
  array([0.5625    , 0.03      , 0.125     , 0.81166667])],
 [[5], array([24]), array([528]), array([0.08])],
 [[6, 6], array([96, 24]), array([120, 144]), array([0.04, 0.21])],
 [[8], array([24]), array([1680]), array([0.03])],
 [[9], array([48]), array([480]), array([0.06])],
 [[10, 10, 10],
  array([48, 72,  6]),
  array([432, 168, 102]),
  array([0.055     , 1.40666667, 0.49333333])],
 [[11, 11, 11, 11, 11, 11],
  array([24, 48, 24, 24, 24, 48]),
  array([420,  96,  96,  48,  24,  96]),
  array([0.03 , 0.125, 0.06 , 0.68 , 0.02 , 0.13 ])],
 [[12, 12, 12, 12],
  array

In [791]:
# %%timeit
def storm_def():
    def calc_min_mit_new(p_events):
        idx, = np.where((np.diff(np.sign(p_events[:,1]-1)) != 0)*1==1)
        if len(idx)==0:
            a = np.abs(p_events[:,1]-1)
            idx, = np.where(a == a.min())
            return p_events[:,0][idx][0]
        return (p_events[:,0][idx][0]+p_events[:,0][idx+1][0])/2.0
    def check_mits_new(mit):
        idx = (bin_events_month  <=-mit)*(bin_events_month<0)
        dry_periods = bin_events_month[idx]*-1
        return np.array([mit, np.std(dry_periods[:]) / np.mean(dry_periods[:]),
                            np.int32(np.mean(dry_periods[:]))])

    # p_data = data[gid]
    p_data[-1] = 1
    v = (p_data==0)*1
    n = v==0
    a = ~n
    c = np.cumsum(a)
    d = np.diff(np.append([0.], c[n]))
    v[n] = -d
    dry_vec = np.cumsum(v)
    bin_events = np.append([1], np.diff(dry_vec))

    event_idx = bin_events<0

    result = []
    for m in [2]:#range(1,13):
        mnth_idx = dt_vec.month == m
        bin_events_month = bin_events[event_idx & mnth_idx]
        # CONDITION 1
        if (len(bin_events_month) == 0):
            # result.append([min_mit, np.nan, np.nan, np.nan])
            continue
        
        # CONDITION 1
        if m<12:
            idx_next_events = np.where((dt_vec.month > m) & (event_idx))
            
            bin_events_next_events = bin_events[idx_next_events]
            bin_events_month = np.append(bin_events_month,bin_events[idx_next_events[0][0]])

            mit_dry = []
            for mit in mit_list:
                _mit = check_mits_new(mit)
                mit_dry.append(_mit)
                if ((_mit[1]<1.0) & (len(mit_dry)>1)):
                # if ( (len(mit_dry)>1)):
                    break    # condition satisfied

            mit_dry = np.array(mit_dry)
            min_mit = calc_min_mit_new(mit_dry)
            
            idx = (bin_events  <= -min_mit) * (bin_events < 0) * (dt_vec.month == m)
            date_idx = np.where(idx)[0]
        
            
            date_idx_month = np.append(date_idx,idx_next_events[0][0])
            dry_periods = bin_events[date_idx_month]*-1
            
            event_durations = date_idx_month[1:] - dry_periods[1:] -  date_idx_month[0:-1]
            event_indices = date_idx_month[0:-1] + event_durations
            n_events = len(event_durations)
            # storm_def_new(data, mit)

            p_totals = [0]*n_events
            for i in range(n_events):
                p_totals[i] = np.sum(p_data[date_idx[i]:event_indices[i]+1])
            result.append([[year]*n_events, [m]*n_events, event_durations, dry_periods[0:-1], p_totals/event_durations])
        if m==12:
            mit_dry = []
            for mit in mit_list:
                _mit = check_mits_new(mit)
                mit_dry.append(_mit)
                if ((_mit[1]<1.0) & (len(mit_dry)>1)):
                    break    # condition satisfied
            mit_dry = np.array(mit_dry)
            min_mit = calc_min_mit_new(mit_dry)
            

            idx = (bin_events  <= -min_mit) * (bin_events < 0) * (dt_vec.month == m)
            date_idx_month = np.where(idx)[0]
            dry_periods = bin_events[date_idx_month]*-1
            
            date_idx = np.where(idx)[0]
            event_durations = date_idx[1:] -dry_periods[1:] - date_idx[0:-1]
            event_indices = date_idx[0:-1] + event_durations
            n_events = len(event_durations)
            # storm_def_new(data, mit)

            p_totals = [0]*n_events
            for i in range(n_events):
                p_totals[i] = np.sum(p_data[date_idx[i]:event_indices[i]+1])
            result.append([[year]*n_events, [m]*n_events, event_durations, dry_periods[0:-1], p_totals/event_durations])
    return result

In [792]:
result = storm_def()

In [790]:
event_indices

array([ 901, 1165, 1381])

In [560]:
idx

array([False, False, False, ..., False, False, False])

In [759]:
result

[[[2009, 2009, 2009],
  [1, 1, 1],
  array([ 48,  24, 144]),
  array([ 13,  48, 360]),
  array([0.38      , 0.09      , 0.68666667])],
 [[2009, 2009, 2009],
  [2, 2, 2],
  array([ 48, 240, 144]),
  array([216,  24,  72]),
  array([0.205, 0.56 , 0.465])],
 [[2009, 2009, 2009, 2009],
  [3, 3, 3, 3],
  array([46, 48, 24, 48]),
  array([ 56,  18, 216, 120]),
  array([2.87173913, 0.405     , 0.03      , 0.7       ])],
 [[2009, 2009, 2009, 2009],
  [4, 4, 4, 4],
  array([ 96,  24,  48, 144]),
  array([360,  72, 192, 120]),
  array([0.5625    , 0.03      , 0.125     , 0.81166667])],
 [[2009], [5], array([24]), array([528]), array([0.08])],
 [[2009, 2009],
  [6, 6],
  array([96, 24]),
  array([120, 144]),
  array([0.04, 0.21])],
 [[2009], [8], array([24]), array([1680]), array([0.03])],
 [[2009], [9], array([48]), array([480]), array([0.06])],
 [[2009, 2009, 2009],
  [10, 10, 10],
  array([48, 72,  6]),
  array([432, 168, 102]),
  array([0.055     , 1.40666667, 0.49333333])],
 [[2009, 2009, 20

# Poisson parameters
- ## $v$ = Number of storms/month
- ## $m_{t0}$ = Mean storm duration (hr)
- ## $m_i$ = Mean storm intensity (mm/hr)
- ## $m_{tb}$ = Mean time between storms (hr)
- ## $\kappa$ = Shape parameter for gamma distribution of storm depth (-)
- ## $\lambda$ = Scale parameter for gamma distribution of storm depth (mm $^{-1}$)
- ## $cov[i, t_r]$ = Covariance of storm intensity and storm duration

In [248]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [833]:
drys = pd.DataFrame({'dt':dt_vec, 'dry':bin_events})
# Create figure with secondary y-axis
fig = go.Figure()
for gid in list(data.keys()):
    # precip = pd.DataFrame({'dt':[x for x in range(len(data[gid]))], 'p':data[gid]})
    precip = pd.DataFrame({'dt':dt_vec, 'p':data[gid]})
    # precip = pd.DataFrame({'dt':range(len(data[gid])), 'p':data[gid]})
    # fig.add_trace(go.Scatter(
    #     name="Raw Data",
    #     mode="markers", x=summary[gid]["dt"], y=summary[gid]["t0"],
    #     marker_symbol="star",yaxis="y2"
    # ))
    fig.add_trace(go.Scatter(
        name="precip",
        mode="lines", x=precip["dt"], y=precip["p"],
        marker_symbol="star", line=dict(width=1),yaxis="y1"
    ))
    # fig.add_trace(go.Scatter(
    #     name="Raw Data",
    #     mode="markers", x=drys["dt"], y=drys["dry"],
    #     marker_symbol="circle",yaxis="y2"
    # ))

fig.update_layout(plot_bgcolor="white",yaxis=dict(
        title="yaxis title",
        titlefont=dict(
            color="#1f77b4"
        ),
        tickfont=dict(
            color="#1f77b4"
        )
    ),
    yaxis2=dict(
        title="yaxis2 title",
        # titlefont=dict(
        #     color="#ff7f0e"
        # ),
        # tickfont=dict(
        #     color="#ff7f0e"
        # ),
        anchor="free",
        overlaying="y",
        side="right",
        position=1
    ),)
        
fig.show()
# plt.plot(data)
# plt.plot(dry_periods)

In [168]:

# Create figure with secondary y-axis
fig = go.Figure()
for gid in list(data.keys()):
    precip = pd.DataFrame({'dt':pd.date_range(start=start_dt, end=end_dt, freq='60min',closed='left')
, 'p':data[gid]})
    fig.add_trace(go.Scatter(
        name="Raw Data",
        mode="markers", x=summary[gid]["dt"], y=summary[gid]["t0"],
        marker_symbol="star",yaxis="y2"
    ))
    fig.add_trace(go.Scatter(
        name="precip",
        mode="lines", x=precip["dt"], y=precip["p"],
        marker_symbol="star", line=dict(width=1),yaxis="y1"
    ))
    fig.add_trace(go.Scatter(
        name="Raw Data",
        mode="markers", x=summary[gid]["dt"], y=summary[gid]["intensity"],
        marker_symbol="circle",yaxis="y1"
    ))

fig.update_layout(plot_bgcolor="white",yaxis=dict(
        title="yaxis title",
        titlefont=dict(
            color="#1f77b4"
        ),
        tickfont=dict(
            color="#1f77b4"
        )
    ),
    yaxis2=dict(
        title="yaxis2 title",
        # titlefont=dict(
        #     color="#ff7f0e"
        # ),
        # tickfont=dict(
        #     color="#ff7f0e"
        # ),
        anchor="free",
        overlaying="y",
        side="right",
        position=1
    ),)
        
fig.show()
# plt.plot(data)
# plt.plot(dry_periods)

In [43]:
mit_min


{104330471: 96.0, 104330475: 36.0}